In [1]:
!pip install --quiet flask langgraph flask-ngrok python-dotenv

In [2]:
import os
folders = ["agent", "templates", "static", "outputs"]
for f in folders:
    os.makedirs(f, exist_ok=True)
print("Folders created:", folders)


Folders created: ['agent', 'templates', 'static', 'outputs']


In [3]:
import os
folders = ["agent", "templates", "static", "outputs"]
for f in folders:
    os.makedirs(f, exist_ok=True)
print("Folders created:", folders)


Folders created: ['agent', 'templates', 'static', 'outputs']


In [4]:
%%writefile templates/index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>AI Agent — Milestone 1 Demo</title>
  <style>
    body{font-family:Inter, Arial, sans-serif;background:#f4f7fb;padding:36px;}
    .card{background:white;padding:22px;border-radius:12px;box-shadow:0 6px 20px rgba(10,20,50,0.08);width:480px}
    h1{color:#0b5bd7;margin:0 0 12px 0}
    .btn{display:inline-block;padding:10px 16px;border-radius:8px;background:#0b5bd7;color:white;border:none;cursor:pointer}
    .field{width:100%;padding:10px;margin:8px 0;border-radius:8px;border:1px solid #e2e8f0}
    #status{margin-top:12px;font-weight:600;color:green}
  </style>
</head>
<body>
  <div class="card">
    <h1>AI Agent Test Page</h1>


    <input id="username" class="field" placeholder="username" />
    <input id="password" class="field" type="password" placeholder="password" />
    <button id="login" class="btn" onclick="doLogin()">Login</button>

    <p id="status"></p>
  </div>

  <script>
    function doLogin(){
      // tiny demo: show success text after "submit"
      document.getElementById('status').innerText = "Login Successful!";
    }
  </script>
</body>
</html>


Writing templates/index.html


In [5]:
from IPython.display import HTML, display
display(HTML(open("templates/index.html","r").read()))


In [6]:
%%writefile app.py
from flask import Flask, render_template
app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")

if __name__ == "__main__":
    app.run()


Writing app.py


In [17]:

from google.colab import output
from app import app

print("Starting frontend preview (iframe) on port 5000...")
output.serve_kernel_port_as_iframe(5000, path='/')

app.run(host="0.0.0.0", port=5000)


Starting frontend preview (iframe) on port 5000...


<IPython.core.display.Javascript object>

 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Dec/2025 09:53:13] "GET / HTTP/1.1" 200 -


In [8]:
%%writefile agent/state.py
from typing import TypedDict, Any, List

class AgentState(TypedDict, total=False):
    instruction: str
    actions: List[dict]
    generated_code: str
    logs: Any
    success: bool
    screenshot: str


Writing agent/state.py


In [9]:
%%writefile agent/text_cleaner.py
import re

class TextCleaner:
    def clean(self, text:str) -> str:
        if not text:
            return ""
        t = text.strip()
        t = re.sub(r"\s+", " ", t)
        return t


Writing agent/text_cleaner.py


In [10]:
%%writefile agent/parser_module.py
from typing import List, Dict

class ParserModule:
    """
    Rule-based parser for Milestone 1. Extendable.
    """
    def parse_instruction(self, instruction: str) -> Dict[str, List[dict]]:
        instr = (instruction or "").lower()
        actions = []

        if "open" in instr or "home" in instr:
            actions.append({"action":"open_url", "url":"http://localhost:5000"})

        if "click login" in instr or "open login" in instr or "login" in instr:
            actions.append({"action":"click", "selector":"#login"})

        if "type username" in instr or "enter username" in instr:
            actions.append({"action":"fill", "selector":"#username", "value":"testuser"})

        if "type password" in instr or "enter password" in instr:
            actions.append({"action":"fill", "selector":"#password", "value":"pass123"})

        if "submit" in instr or "click submit" in instr:
            actions.append({"action":"click", "selector":"#login"})  # login button as submit

        if "assert" in instr or "success" in instr:
            actions.append({"action":"assert_text", "selector":"#status", "value":"Login Successful"})

        return {"actions": actions}


Writing agent/parser_module.py


In [11]:
%%writefile agent/code_generator.py
from typing import List

class CodeGenerator:
    """
    Baseline generator: produces a small Python function (pseudo/print/log style)
    that the simple executor can run safely inside Colab.
    """
    def generate_code(self, actions: List[dict]) -> str:
        lines = []
        lines.append("def run_test():")
        lines.append("    logs = []")
        for a in actions:
            if a["action"] == "open_url":
                lines.append(f"    logs.append('OPEN {a.get('url','') }')")
            elif a["action"] == "click":
                lines.append(f"    logs.append('CLICK {a.get('selector','') }')")
            elif a["action"] == "fill":
                lines.append(f\"    logs.append('FILL {a.get('selector','')} = {a.get('value','')}')\")
            elif a["action"] == "assert_text":
                lines.append(f\"    logs.append('ASSERT {a.get('selector','')} contains {a.get('value','')}')\")
            else:
                lines.append(f\"    logs.append('UNKNOWN {a}')\")
        lines.append("    return logs")
        return \"\\n\".join(lines)

# Also write a Playwright-capable generator (optional to run)
%%bash
cat > agent/code_generator_playwright.py <<'PY'
from typing import List
class PlaywrightGenerator:
    """
    Generates a Playwright sync-api Python script string for the given actions.
    NOTE: To run generated Playwright code you must have Playwright + chromium installed.
    """
    def generate_code(self, actions: List[dict]) -> str:
        lines = []
        lines.append("from playwright.sync_api import sync_playwright")
        lines.append("def run_test(base_url):")
        lines.append("    logs = []")
        lines.append("    with sync_playwright() as p:")
        lines.append("        browser = p.chromium.launch(headless=True)")
        lines.append("        page = browser.new_page()")
        for a in actions:
            if a["action"] == "open_url":
                lines.append("        page.goto(base_url)")
                lines.append("        logs.append(f'OPEN {base_url}')")
            elif a["action"] == "click":
                lines.append(f"        page.click('{a.get('selector','')}')")
                lines.append(f"        logs.append('CLICK {a.get('selector','')}')")
            elif a["action"] == "fill":
                lines.append(f"        page.fill('{a.get('selector','')}', '{a.get('value','')}')")
                lines.append(f"        logs.append('FILL {a.get('selector','')}')")
            elif a["action"] == "assert_text":
                lines.append(f"        text = page.inner_text('{a.get('selector','')}')")
                lines.append(f"        assert '{a.get('value','')}' in text")
                lines.append(f"        logs.append('ASSERT OK')")
        lines.append("        browser.close()")
        lines.append("    return logs")
        return '\\n'.join(lines)
PY


Writing agent/code_generator.py


In [12]:
%%writefile agent/executor.py
import traceback
from agent.timeline import Timeline

class Executor:
    """
    Executes generated code produced by the baseline CodeGenerator safely.
    It executes a local function `run_test()` and returns logs and success flag.
    """
    def execute(self, generated_code: str) -> dict:
        tl = Timeline()
        tl.record("start")
        try:
            ns = {}
            exec(generated_code, {}, ns)
            tl.record("compiled")
            if "run_test" in ns and callable(ns["run_test"]):
                res = ns["run_test"]()
                tl.record("executed")
                return {"logs": res, "success": True, "timeline": tl.get()}
            else:
                return {"logs": ["run_test missing"], "success": False, "timeline": tl.get()}
        except Exception as e:
            return {"logs": [f"Exception: {e}", traceback.format_exc()], "success": False, "timeline": tl.get()}


Writing agent/executor.py


In [13]:
%%writefile agent/workflow.py
from langgraph.graph import StateGraph, END
from agent.state import AgentState
from agent.text_cleaner import TextCleaner
from agent.parser_module import ParserModule
from agent.code_generator import CodeGenerator
from agent.executor import Executor

cleaner = TextCleaner()
parser = ParserModule()
generator = CodeGenerator()
executor = Executor()

def parser_node(state: AgentState):
    instr = state.get("instruction", "")
    instr = cleaner.clean(instr)
    parsed = parser.parse_instruction(instr)
    return {"actions": parsed["actions"], "instruction": instr}

def codegen_node(state: AgentState):
    actions = state.get("actions", [])
    code = generator.generate_code(actions)
    return {"generated_code": code}

def executor_node(state: AgentState):
    code = state.get("generated_code", "")
    result = executor.execute(code)
    return {"logs": result.get("logs"), "success": result.get("success"), "timeline": result.get("timeline")}

graph = StateGraph(AgentState, input_keys=["instruction"])
graph.add_node("parse", parser_node)
graph.add_node("codegen", codegen_node)
graph.add_node("execute", executor_node)
graph.set_entry_point("parse")
graph.add_edge("parse", "codegen")
graph.add_edge("codegen", "execute")
graph.add_edge("execute", END)
workflow = graph.compile()
print("LangGraph workflow (Milestone 1) ready")


Writing agent/workflow.py


In [14]:
%%writefile agent/code_generator.py
from typing import List

class CodeGenerator:
    """
    Baseline generator: produces a small Python function (pseudo/print/log style)
    that the simple executor can run safely inside Colab.
    """
    def generate_code(self, actions: List[dict]) -> str:
        lines = []
        lines.append("def run_test():")
        lines.append("    logs = []")

        for a in actions:
            action = a.get("action", "")

            if action == "open_url":
                url = a.get("url", "")
                lines.append(f"    logs.append('OPEN {url}')")

            elif action == "click":
                selector = a.get("selector", "")
                lines.append(f"    logs.append('CLICK {selector}')")

            elif action == "fill":
                selector = a.get("selector", "")
                value = a.get("value", "")
                lines.append(f"    logs.append('FILL {selector} = {value}')")

            elif action == "assert_text":
                selector = a.get("selector", "")
                value = a.get("value", "")
                lines.append(f"    logs.append('ASSERT {selector} contains {value}')")

            else:
                lines.append(f"    logs.append('UNKNOWN ACTION: {a}')")

        lines.append("    return logs")
        return "\n".join(lines)


Overwriting agent/code_generator.py


In [15]:
%%writefile agent/code_generator_playwright.py
from typing import List

class PlaywrightGenerator:
    """
    Generates a Playwright sync-api Python script string for the given actions.
    NOTE: To run generated Playwright code, you must have Playwright + chromium installed.
    """

    def generate_code(self, actions: List[dict]) -> str:
        lines = []
        lines.append("from playwright.sync_api import sync_playwright")
        lines.append("")
        lines.append("def run_test(base_url):")
        lines.append("    logs = []")
        lines.append("    with sync_playwright() as p:")
        lines.append("        browser = p.chromium.launch(headless=True)")
        lines.append("        page = browser.new_page()")

        for a in actions:
            action = a.get("action", "")

            if action == "open_url":
                lines.append("        page.goto(base_url)")
                lines.append("        logs.append(f'OPEN {base_url}')")

            elif action == "click":
                selector = a.get("selector", "")
                lines.append(f"        page.click('{selector}')")
                lines.append(f"        logs.append('CLICK {selector}')")

            elif action == "fill":
                selector = a.get("selector", "")
                value = a.get("value", "")
                lines.append(f"        page.fill('{selector}', '{value}')")
                lines.append(f"        logs.append('FILL {selector} = {value}')")

            elif action == "assert_text":
                selector = a.get("selector", "")
                value = a.get("value", "")
                lines.append(f"        text = page.inner_text('{selector}')")
                lines.append(f"        assert '{value}' in text")
                lines.append("        logs.append('ASSERT OK')")

            else:
                lines.append(f"        logs.append('UNKNOWN ACTION: {a}')")

        lines.append("        browser.close()")
        lines.append("    return logs")

        return \"\n\".join(lines)


Writing agent/code_generator_playwright.py


In [16]:

from IPython.display import Markdown, display
display(Markdown("**Workflow:** Parse → CodeGen → Execute → END"))


**Workflow:** Parse → CodeGen → Execute → END